In [2]:
from pathlib import Path
import pandas as pd
# import yfinance as yf
import numpy as np
import sklearn as sk
import hvplot.pandas
from watermark import watermark
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 2000)

In [5]:
#System Requirements

print(watermark())
print(watermark(iversions=True, globals_=globals()))

Last updated: 2022-12-08T20:31:47.578855-05:00

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 7.31.1

Compiler    : Clang 12.0.0 
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

numpy  : 1.21.5
hvplot : 0.8.0
sklearn: 0.0.post1
pandas : 1.3.5



In [3]:
# Read data from the Kaggle database file into a Pandas DataFrame
df = pd.read_csv(
    Path("../../LG_Resources/Resources/lending-club/accepted_2007_to_2018Q4.csv.gz"),  
    infer_datetime_format=True,
    compression='gzip', 
    low_memory=False
)


FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\LG_Resources\\Resources\\lending-club\\accepted_2007_to_2018Q4.csv.gz'

In [136]:
# Review the DataFrame
display(df.shape)
df.head()

(2260701, 151)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
# column_names = df.columns.tolist()
# df['loan_amnt'].hvplot.bar(x = df['loan_amnt'].value_counts(), y=df['loan_amnt'])

In [138]:
nan_values = pd.DataFrame(df.isna().sum(),columns = ["NAN Count"]).reset_index()
nan_values.sort_values(nan_values.columns[1], ascending=False)

,index,NAN Count
1,member_id,2260701
140,orig_projected_additional_accrued_interest,2252050
135,hardship_end_date,2249784
134,hardship_start_date,2249784
129,hardship_type,2249784
130,hardship_reason,2249784
131,hardship_status,2249784
132,deferral_term,2249784
142,hardship_last_payment_amount,2249784
141,hardship_payoff_balance_amount,2249784


In [144]:
nan_values[(nan_values['NAN Count'] > 500000)].shape
drop_columns = nan_values[(nan_values['NAN Count'] > 500000)]['index'].tolist()
drop_columns.append('id')
df_1 = df.drop(drop_columns, axis=1)
df_1

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,...,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,...,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,...,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,N,Cash,N
3,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,...,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,N,Cash,N
4,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,...,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,40000.0,40000.0,40000.0,60 months,10.49,859.56,B,B3,Vice President,9 years,...,50.0,0.0,0.0,55970.0,28398.0,12300.0,42670.0,N,Cash,N
2260697,24000.0,24000.0,24000.0,60 months,14.49,564.56,C,C4,Program Manager,6 years,...,40.0,1.0,0.0,84664.0,62426.0,20700.0,58764.0,N,Cash,Y
2260698,14000.0,14000.0,14000.0,60 months,14.49,329.33,C,C4,Customer Service Technician,10+ years,...,50.0,0.0,0.0,163804.0,44215.0,9500.0,34169.0,N,Cash,N
2260699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
